In [20]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.metrics import mean_squared_error,r2_score,mean_absolute_error,root_mean_squared_error
from sklearn.neighbors import KNeighborsRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor,AdaBoostRegressor
from sklearn.svm import SVR
from sklearn.linear_model import LinearRegression, Ridge, Lasso
from sklearn.model_selection import RandomizedSearchCV
from xgboost import XGBRegressor
from catboost import CatBoostRegressor
import warnings

warnings.filterwarnings('ignore')


In [2]:
df = pd.read_csv('data/modified_study.csv')

In [3]:
df.head()

,Unnamed: 0,gender,race_ethnicity,parental_level_of_education,lunch,test_preparation_course,math_score,reading_score,writing_score,total_score,average
0,0,female,group B,bachelor's degree,standard,none,72,72,74,218,72.666667
1,1,female,group C,some college,standard,completed,69,90,88,247,82.333333
2,2,female,group B,master's degree,standard,none,90,95,93,278,92.666667
3,3,male,group A,associate's degree,free/reduced,none,47,57,44,148,49.333333
4,4,male,group C,some college,standard,none,76,78,75,229,76.333333


In [11]:
# splitting x and y datasets

In [6]:
df.drop(columns=['average'],axis=1,inplace=True)

In [4]:
df.drop(columns='Unnamed: 0',axis=1,inplace=True)

In [7]:
df.head()

,gender,race_ethnicity,parental_level_of_education,lunch,test_preparation_course,math_score,reading_score,writing_score,total_score
0,female,group B,bachelor's degree,standard,none,72,72,74,218
1,female,group C,some college,standard,completed,69,90,88,247
2,female,group B,master's degree,standard,none,90,95,93,278
3,male,group A,associate's degree,free/reduced,none,47,57,44,148
4,male,group C,some college,standard,none,76,78,75,229


In [8]:
x=df.drop(columns='total_score',axis=1)

In [9]:
y=df['total_score']

In [10]:
y.head()

0    218
1    247
2    278
3    148
4    229
Name: total_score, dtype: int64

In [11]:
# create column transformer for 3 types of transformation
num_features = x.select_dtypes(exclude='object').columns
cat_features = x.select_dtypes(include='object').columns

from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.compose import ColumnTransformer

numeric_transformer = StandardScaler()
cat_transformer = OneHotEncoder()

preprocessor = ColumnTransformer(
         [
                  ("OneHotEncoder",cat_transformer,cat_features),
                  ("Standard Scaler",numeric_transformer,num_features)
         ]
)


In [12]:
x = preprocessor.fit_transform(x)

In [14]:
x

array([[ 1.        ,  0.        ,  0.        , ...,  0.39002351,
         0.19399858,  0.39149181],
       [ 1.        ,  0.        ,  0.        , ...,  0.19207553,
         1.42747598,  1.31326868],
       [ 1.        ,  0.        ,  0.        , ...,  1.57771141,
         1.77010859,  1.64247471],
       ...,
       [ 1.        ,  0.        ,  0.        , ..., -0.46775108,
         0.12547206, -0.20107904],
       [ 1.        ,  0.        ,  0.        , ...,  0.12609287,
         0.60515772,  0.58901542],
       [ 1.        ,  0.        ,  0.        , ...,  0.71993682,
         1.15336989,  1.18158627]])

In [15]:
x.shape

(1000, 20)

In [16]:
from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test = train_test_split(x,y,test_size=0.25,random_state=42)


In [17]:
x_train.shape,x_test.shape


((750, 20), (250, 20))

In [21]:
# Evaluating function to give all metrics after model training

def evalutate_model(true,predicted):
         mae = mean_absolute_error(true,predicted)
         mse = mean_squared_error(true,predicted)
         rmse = root_mean_squared_error(true,predicted)
         r2score = r2_score(true,predicted)

         return mae,mse,rmse,r2score

In [22]:
models = {
         "Linear Regression": LinearRegression(),
         "Lasso": Lasso(),
         "Ridge": Ridge(),
         "K-Neighbours Regressor": KNeighborsRegressor(),
         "Decsion Tree": DecisionTreeRegressor(),
         "Random Forest": RandomForestRegressor(),
         "XG Boost": XGBRegressor(),
         "Cat Boost": CatBoostRegressor(),
         "AdaBoost Regressor": AdaBoostRegressor()
}

model_list = []
r2_list = []

for name in models:
         model = models[name]
         model.fit(x_train,y_train)

         y_pred_train = model.predict(x_train)
         y_pred_test = model.predict(x_test)

         train_mae, train_mse, train_rmse, train_r2Score = evalutate_model(y_train,y_pred_train)
         test_mae, test_mse, test_rmse, test_r2Score = evalutate_model(y_test,y_pred_test)

         print(name)
         model_list.append(name)

         print("Model performance for Training set")
         print(f"Mean Absolute Error: {train_mae}")
         print(f"Mean Squared Error: {train_mse}")
         print(f"Root Mean Squared Error: {train_rmse}")
         print(f"R2 Score: {train_r2Score}")

         print('---------------------------------------------')

         print("Model performance for Test set")
         print(f"Mean Absolute Error: {test_mae}")
         print(f"Mean Squared Error: {test_mse}")
         print(f"Root Mean Squared Error: {test_rmse}")
         print(f"R2 Score: {test_r2Score}")

         r2_list.append(test_r2Score)

         print('\n')


Linear Regression
Model performance for Training set
Mean Absolute Error: 4.490630090003833e-15
Mean Squared Error: 1.246694738320481e-28
Root Mean Squared Error: 1.1165548523563368e-14
R2 Score: 1.0
---------------------------------------------
Model performance for Test set
Mean Absolute Error: 5.059064278611913e-15
Mean Squared Error: 1.3974828708171267e-28
Root Mean Squared Error: 1.1821517968590695e-14
R2 Score: 1.0


Lasso
Model performance for Training set
Mean Absolute Error: 0.8627748242420049
Mean Squared Error: 1.1570870517993395
Root Mean Squared Error: 1.0756798091436595
R2 Score: 0.999340249196846
---------------------------------------------
Model performance for Test set
Mean Absolute Error: 0.9065681773119516
Mean Squared Error: 1.3498198153591865
Root Mean Squared Error: 1.1618174621510844
R2 Score: 0.9993296257319948


Ridge
Model performance for Training set
Mean Absolute Error: 0.020908024897098205
Mean Squared Error: 0.0006682309642072351
Root Mean Squared Error: 

In [23]:
pd.DataFrame(list(zip(model_list, r2_list)),columns=['Model Name', 'R2 Score']).sort_values(by=['R2 Score'],ascending=False)

,Model Name,R2 Score
0,Linear Regression,1.000000
2,Ridge,1.000000
1,Lasso,0.999330
6,XG Boost,0.994129
7,Cat Boost,0.993779
5,Random Forest,0.992809
4,Decsion Tree,0.988009
8,AdaBoost Regressor,0.973685
3,K-Neighbours Regressor,0.956667
